In [1]:
docker ps --filter="name=sql"

CONTAINER ID   IMAGE                                   COMMAND                  CREATED       STATUS       PORTS                    NAMES
538ebf573fff   mcr.microsoft.com/mssql/server:latest   "/opt/mssql/bin/perm…"   2 hours ago   Up 2 hours   0.0.0.0:1433->1433/tcp   sql


In [2]:
if (!$OriginalProjectPath) {$OriginalProjectPath = (Get-Item .).FullName}
$WorkingDirectory = $OriginalProjectPath + '\WorkingDirectory\LocalDockerExample'
"OriginalProjectPath: $OriginalProjectPath"
"WorkingDirectory: $WorkingDirectory"

OriginalProjectPath: C:\Users\gary\OneDrive\Documents\GitHub\terraform-101
WorkingDirectory: C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalDockerExample


In [3]:
cd $env:TEMP\tf
$TerraformExePath = (Get-ChildItem)[0].FullName


In [8]:
"TerraformExePath: $TerraformExePath"

TerraformExePath: C:\Users\gary\AppData\Local\Temp\tf\terraform.exe


In [4]:
.\terraform version

Terraform v1.4.2
on windows_amd64


In [5]:
$command = 'docker inspect --format="{{.ID}}" sql'
Write-Output "Executing the command:"
Write-Output $command
$DockerId = Invoke-Expression $command
Write-Output 'Docker Id in question:' $DockerId

Executing the command:
docker inspect --format="{{.ID}}" sql
Docker Id in question:
538ebf573fff2b57c9558864f6361bfed89fb8e47104235ca0bb13e15c8469f8


In [9]:
$command = ".\terraform -chdir=$WorkingDirectory init"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

Executing the command:
.\terraform -chdir=C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalDockerExample init

Initializing the backend...

Initializing provider plugins...
- Reusing previous version of kreuzwerker/docker from the dependency lock file
- Using previously-installed kreuzwerker/docker v2.17.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory import docker_container.arbitrarylabel $DockerId"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [11]:
$command = ".\terraform -chdir=$WorkingDirectory show"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

Executing the command:
.\terraform -chdir=C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalDockerExample show
# docker_container.arbitrarylabel:
resource "docker_container" "arbitrarylabel" {
    command           = [
        "/opt/mssql/bin/sqlservr",
    ]
    cpu_shares        = 0
    dns               = []
    dns_opts          = []
    dns_search        = []
    entrypoint        = [
        "/opt/mssql/bin/permissions_check.sh",
    ]
    exit_code         = 137
    group_add         = []
    hostname          = "538ebf573fff"
    id                = "538ebf573fff2b57c9558864f6361bfed89fb8e47104235ca0bb13e15c8469f8"
    image             = "sha256:880c02b9631c5813e3e5d0ed8cb0aaca6db059311d41d199dd0409b7850410ef"
    init              = false
    ip_prefix_length  = 0
    ipc_mode          = "private"
    links             = []
    log_driver        = "json-file"
    log_opts          = {}
    max_retry_count   = 0
    memory            = 0
    memory_sw

In [14]:
$command = ".\terraform -chdir=$WorkingDirectory plan"
Write-Output "Executing the command:"
Write-Output $command
Write-Output "****** Errors are Expected ******"
Invoke-Expression $command

Executing the command:
.\terraform -chdir=C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalDockerExample plan
****** Errors are Expected ******
╷
│ Error: Missing required argument
│ 
│   on LocalDockerExample.tf line 13, in resource "docker_container" "arbitrarylabel":
│   13: resource "docker_container" "arbitrarylabel" {
│ 
│ The argument "image" is required, but no definition was found.
╵
╷
│ Error: Missing required argument
│ 
│   on LocalDockerExample.tf line 13, in resource "docker_container" "arbitrarylabel":
│   13: resource "docker_container" "arbitrarylabel" {
│ 
│ The argument "name" is required, but no definition was found.
╵


In [15]:
$command = ".\terraform -chdir=$WorkingDirectory show -no-color > $WorkingDirectory\LocalDockerExample.tf"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

Executing the command:
.\terraform -chdir=C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalDockerExample show -no-color > C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalDockerExample\LocalDockerExample.tf
****** Errors are Expected ******


In [24]:

@'
terraform {
    required_providers {
        docker = {
            source  = "kreuzwerker/docker"
            version = "~> 2.17.0"
        }
    }
}
provider "docker" {
  #host = "unix:///var/run/docker.sock"
  host    = "npipe:////.//pipe//docker_engine"
}
'@ + (Get-Content "$WorkingDirectory\LocalDockerExample.tf") | Set-Content "$WorkingDirectory\LocalDockerExample.tf"

terraform {
    required_providers {
        docker = {
            source  = "kreuzwerker/docker"
            version = "~> 2.17.0"
        }
    }
}
provider "docker" {
  #host = "unix:///var/run/docker.sock"
  host    = "npipe:////.//pipe//docker_engine"
}terraform {     required_providers {         docker = {             source  = "kreuzwerker/docker"             version = "~> 2.17.0"         }     } } provider "docker" {   #host = "unix:///var/run/docker.sock"   host    = "npipe:////.//pipe//docker_engine" } # docker_container.arbitrarylabel: resource "docker_container" "arbitrarylabel" {     command           = [         "/opt/mssql/bin/sqlservr",     ]     cpu_shares        = 0     dns               = []     dns_opts          = []     dns_search        = []     entrypoint        = [         "/opt/mssql/bin/permissions_check.sh",     ]     exit_code         = 137     group_add         = []     hostname          = "538ebf573fff"     id                = "538ebf573fff2b57c9558864f63

In [19]:
$command = ".\terraform -chdir=$WorkingDirectory init -upgrade"
Write-Output "Executing the command:"
Write-Output $command
Write-Output "****** Errors are Expected ******"
Invoke-Expression $command

Executing the command:
.\terraform -chdir=C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalDockerExample init -upgrade
****** Errors are Expected ******

Initializing the backend...

Initializing provider plugins...
- Finding kreuzwerker/docker versions matching "~> 2.17.0"...
- Using previously-installed kreuzwerker/docker v2.17.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [30]:
$command = ".\terraform -chdir=$WorkingDirectory plan"
Write-Output "Executing the command:"
Write-Output $command
Write-Output "****** Errors are NOT Expected ******"
Invoke-Expression $command

Executing the command:
.\terraform -chdir=C:\Users\gary\OneDrive\Documents\GitHub\terraform-101\WorkingDirectory\LocalDockerExample plan
****** Errors are NOT Expected ******
╷
│ Warning: Argument is deprecated
│ 
│   with docker_container.arbitrarylabel,
│   on LocalDockerExample.tf line 33, in resource "docker_container" "arbitrarylabel":
│   33:     links             = []
│ 
│ The --link flag is a legacy feature of Docker. It may eventually be
│ removed.
╵
╷
│ Error: Invalid or unknown key
│ 
│   with docker_container.arbitrarylabel,
│   on LocalDockerExample.tf line 28, in resource "docker_container" "arbitrarylabel":
│   28:     id                = "538ebf573fff2b57c9558864f6361bfed89fb8e47104235ca0bb13e15c8469f8"
│ 
╵
╷
│ Error: Value for unconfigurable attribute
│ 
│   with docker_container.arbitrarylabel,
│   on LocalDockerExample.tf line 31, in resource "docker_container" "arbitrarylabel":
│   31:     ip_prefix_length  = 0
│ 
│ Can't configure a value for "ip_prefix_length": i

In [27]:
$content = Get-Content -Path "$WorkingDirectory\LocalDockerExample.tf"
$content = $content.Replace('# exit_code','exit_code').Replace('exit_code','# exit_code')
$content | Set-Content "$WorkingDirectory\LocalDockerExample.tf"

In [ ]:
## Full Reset
if (!$WorkingDirectory) {exit} # Guard Clause

$command = "rm $WorkingDirectory\.terraform"
Write-Output "Executing the command:"
Write-Output $command
Write-Output "****** Errors are NOT Expected ******"
Invoke-Expression $command